# ETL
### Introduction to the Datasets

In this data science project, we have access to three JSON files: `output_steam_games.json`, `australian_users_items.json`, and `australian_user_reviews.json`. These datasets are essential components of our analysis and are centered around Steam, a widely-used platform for gaming enthusiasts. Let's briefly introduce each dataset:

1. **output_steam_games.json**: This dataset contains comprehensive information about the games available on Steam. It encompasses details such as game titles, genres, release dates,etc... allowing us to gain insights into the gaming catalog.

2. **users_items.json**: Within this dataset, we find data related to Steam users and their collections of in-game items. By studying this dataset, we can uncover user preferences and the specific items they possess.

3. **user_reviews.json**: The user reviews dataset offers a treasure trove of information, consisting of user-generated reviews and feedback for Steam games. This dataset includes vital components like review text,  and helpfulness votes, that will enabling us to gauge user sentiments and experiences.


In [2]:
import json
import ast
import pandas as pd
import re
import pyarrow
import numpy as np


# output_steam_games

In [3]:
# load dataset
df_games = pd.read_json('../data/raw/output_steam_games.json', lines=True)
df_games.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,discount_price,specs,price,early_access,id,metascore,developer,user_id,steam_id,items,items_count
0,None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,None,None,76561197970982479,7.656120e+16,"[{'item_id': '10', 'item_name': 'Counter-Strik...",277.0
1,None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,None,None,js41637,7.656120e+16,"[{'item_id': '10', 'item_name': 'Counter-Strik...",888.0
2,None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,None,None,evcentric,7.656120e+16,"[{'item_id': '1200', 'item_name': 'Red Orchest...",137.0
3,None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,None,None,Riot-Punch,7.656120e+16,"[{'item_id': '10', 'item_name': 'Counter-Strik...",328.0
4,None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,None,None,doctr,7.656120e+16,"[{'item_id': '300', 'item_name': 'Day of Defea...",541.0


In [4]:
df_games.tail(3)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,discount_price,specs,price,early_access,id,metascore,developer,user_id,steam_id,items,items_count
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,1.39,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,None,Laush Dmitriy Sergeevich,None,NaN,None,NaN
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,NaN,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,None,"xropi,stev3ns",None,NaN,None,NaN
120444,None,None,Maze Run VR,None,http://store.steampowered.com/app/681550/Maze_...,None,"[Early Access, Adventure, Indie, Action, Simul...",http://steamcommunity.com/app/681550/reviews/?...,NaN,"[Single-player, Stats, Steam Leaderboards, HTC...",4.99,1.0,681550.0,None,None,None,NaN,None,NaN


In [5]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   publisher       24083 non-null  object 
 1   genres          28852 non-null  object 
 2   app_name        32133 non-null  object 
 3   title           30085 non-null  object 
 4   url             32135 non-null  object 
 5   release_date    30068 non-null  object 
 6   tags            31972 non-null  object 
 7   reviews_url     32133 non-null  object 
 8   discount_price  225 non-null    float64
 9   specs           31465 non-null  object 
 10  price           30758 non-null  object 
 11  early_access    32135 non-null  float64
 12  id              32133 non-null  float64
 13  metascore       2677 non-null   object 
 14  developer       28836 non-null  object 
 15  user_id         88310 non-null  object 
 16  steam_id        88310 non-null  float64
 17  items           88310 non-nul

In [6]:
# Dropping Unnecessary Columns 
df_games.drop(columns=['publisher',
                       'title'
                       'url',
                       'reviews_url',
                       'discount_price',
                       'specs',
                       'early_access',
                       'metascore',
                       'developer',
                       'user_id',
                       'steam_id',
                       'items',
                       'items_count'], inplace=True)

In [8]:
# Drop rows that are all null values
df_games.dropna(how='all', inplace=True)

In [10]:
# Drop rows without 'id'
df_games.dropna(subset=['id'], inplace=True)
df_games.drop_duplicates(subset=['id'], inplace=True)

In [18]:
# drop row without app_name and rename app_name to title
df_games.dropna(subset=['app_name'], inplace=True)
df_games.rename(columns={'app_name':'title'}, inplace=True)

In [22]:
df_games.release_date.unique()

array(['2018-01-04', '2017-07-24', '2017-12-07', ..., '2016-11-19',
       'January 2018', '2018-10-01'], dtype=object)

In [23]:
# create the column year and extract the year because there are differenct date formats(lack of consistency)
df_games['year'] = df_games['release_date'].str.extract(r'(\d{4})')
df_games.year.unique()

array(['2018', '2017', nan, '1997', '1998', '2016', '2006', '2005',
       '2003', '2007', '2002', '2000', '1995', '1996', '1994', '2001',
       '1993', '2004', '1999', '2008', '2009', '1992', '1989', '2010',
       '2011', '2013', '2012', '2014', '1983', '1984', '2015', '1990',
       '1988', '1991', '1985', '1982', '1987', '1981', '1986', '2021',
       '5275', '2019', '1975', '1970', '1980'], dtype=object)

In [24]:
df_games.drop(columns=['release_date'], inplace=True)

In [28]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28730 entries, 88310 to 120443
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   genres  28730 non-null  object 
 1   title   28730 non-null  object 
 2   tags    28706 non-null  object 
 3   price   27613 non-null  object 
 4   id      28730 non-null  float64
 5   year    28730 non-null  object 
dtypes: float64(1), object(5)
memory usage: 1.5+ MB


In [32]:
df_games.price.unique()

array([4.99, 'Free To Play', 'Free to Play', 0.99, 3.99, 9.99, 18.99,
       29.99, None, 10.99, 2.99, 1.5899999999999999, 14.99, 1.99, 59.99,
       8.99, 6.99, 7.99, 39.99, 'Free', 19.99, 7.49, 12.99, 5.99, 2.49,
       15.99, 1.25, 24.99, 17.99, 61.99, 3.49, 11.99, 13.99, 'Free Demo',
       'Play for Free!', 34.99, 1.49, 32.99, 99.99, 14.95, 69.99, 16.99,
       79.99, 49.99, 5.0, 44.99, 13.98, 29.96, 109.99, 149.99, 771.71,
       'Install Now', 21.99, 89.99, 'Play WARMACHINE: Tactics Demo', 0.98,
       139.92, 4.29, 64.99, 'Free Mod', 54.99, 74.99, 'Install Theme',
       0.89, 'Third-party', 0.5, 'Play Now', 299.99, 1.29, 119.99, 3.0,
       15.0, 5.49, 23.99, 49.0, 20.99, 10.93, 1.3900000000000001,
       'Free HITMAN™ Holiday Pack', 36.99, 4.49, 2.0, 4.0,
       1.9500000000000002, 1.5, 199.0, 189.0, 6.66, 27.99, 129.99, 179.0,
       26.99, 399.99, 31.99, 399.0, 20.0, 40.0, 3.33, 22.99, 320.0, 38.85,
       71.7, 995.0, 27.49, 3.39, 6.0, 19.95, 499.99, 199.99, 16.06, 4.68,
 

In [27]:
# drop rows without value in genres
df_games.dropna(subset=['genres', 'year'], inplace=True)

In [39]:
# we need to standarize the price column 

# Convert NaN values to 0
df_games['price'].fillna(0, inplace=True)

# Apply lambda function to replace string values with 0 and round numeric values to integers
df_games['price'] = df_games['price'].apply(lambda x: 0 if isinstance(x, str) else round(x))

df_games.price.unique()




array([  5,   0,   1,   4,  10,  19,  30,  11,   3,   2,  15,  60,   9,
         7,   8,  40,  20,  13,   6,  16,  25,  18,  62,  12,  14,  35,
        33, 100,  70,  17,  80,  50,  45, 110, 150, 772,  22,  90, 140,
        65,  55,  75, 300, 120,  24,  49,  21,  37, 199, 189,  28, 130,
       179,  27, 400,  32, 399,  23, 320,  39,  72, 995, 500, 200, 131,
       203, 172, 250,  99,  88,  43,  42, 290, 190, 125, 161], dtype=int64)

In [41]:
# set id column as string type
df_games['id'] = df_games.id.astype(str)

In [43]:
df_games.reset_index(drop=True, inplace=True)

In [44]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28730 entries, 0 to 28729
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   genres  28730 non-null  object
 1   title   28730 non-null  object
 2   tags    28706 non-null  object
 3   price   28730 non-null  int64 
 4   id      28730 non-null  object
 5   year    28730 non-null  object
dtypes: int64(1), object(5)
memory usage: 1.3+ MB


In [46]:
# save the cleaned dataset
df_games.to_parquet('../data/fixed/games.parquet', engine='pyarrow')

# australian_user_reviews.json

In [3]:
rows = []
with open('../data/raw/australian_user_reviews.json', encoding='utf-8') as f:
    for line in f.readlines():
        rows.append(ast.literal_eval(line))
        
df_reviews = pd.DataFrame(rows)
df_reviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [50]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25485 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25485 non-null  object
 1   user_url  25485 non-null  object
 2   reviews   25485 non-null  object
dtypes: object(3)
memory usage: 796.4+ KB


In [4]:
# drop duplicates
df_reviews.drop_duplicates(subset="user_id", inplace=True)

In [5]:
# cast user_id as string type
df_reviews['user_id'] = df_reviews['user_id'].astype(str)


In [6]:
# need to work with the values in the column review to create the sentiment
# Initialize an empty list to store the unnested reviews
unnested_reviews = []

# Iterate through each row in the 'df_reviews' DataFrame
for index, row in df_reviews.iterrows():
    user_id = row['user_id']
    user_url = row['user_url']
    reviews = row['reviews']
    
    # Iterate through each review in the 'reviews' list
    for review in reviews:
        new_review = {
            'user_id': user_id,
            'user_url': user_url,
            'funny': review.get('funny', ''),
            'posted': review.get('posted', ''),
            'last_edited': review.get('last_edited', ''),
            'item_id': review.get('item_id', ''),
            'helpful': review.get('helpful', ''),
            'recommend': review.get('recommend', ''),
            'review_text': review.get('review', '')  # Renamed 'review' to 'review_text'
        }
        
        # Append the new review to the list
        unnested_reviews.append(new_review)

# Create a new DataFrame 'df_reviews_unnested' from the list of unnested reviews
df_reviews_unnested = pd.DataFrame(unnested_reviews)


In [7]:
df_reviews_unnested.head()

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review_text
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


In [12]:
# with the review_text we proceed to create the sentiment_analysis column: 0 if bad, 1 if neutral and 2 if positive.

# these are the libraries that will help us
from nltk.corpus import stopwords
from textblob import TextBlob
import re


In [9]:
# to create the sentiment_analysis column first we need to eliminate emojis, and special characters 
df_reviews_unnested['review_text'] = df_reviews_unnested['review_text'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
# also need to eliminate the preposition so only the impact words are left behind


In [14]:
stop_words_en = set(stopwords.words('english'))
df_reviews_unnested['review_text'] = df_reviews_unnested['review_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words_en]))

In [16]:
stop_words_es = set(stopwords.words('spanish'))
df_reviews_unnested['review_text'] = df_reviews_unnested['review_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words_es]))

In [17]:
stop_words_pr = set(stopwords.words('portuguese'))
df_reviews_unnested['review_text'] = df_reviews_unnested['review_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words_pr]))

In [19]:
df_reviews_unnested.review_text.head()

0    Simple yet great replayability In opinion zomb...
1                         Its unique worth playthrough
2    Great atmosphere The gunplay bit chunky times ...
3    I know think see title Barbie Dreamhouse Party...
4    For simple actually simple truck driving Simul...
Name: review_text, dtype: object

In [22]:

# Preprocess the 'review_text' column
df_reviews_unnested['cleaned_review'] = df_reviews_unnested['review_text'].str.lower()  # Convert to lowercase
df_reviews_unnested['cleaned_review'] = df_reviews_unnested['cleaned_review'].str.replace('[^a-zA-Z0-9]', ' ')  # Remove special characters

# Perform sentiment analysis
df_reviews_unnested['sentiment_polarity'] = df_reviews_unnested['cleaned_review'].apply(lambda x: TextBlob(x).sentiment.polarity)

# Categorize sentiment
df_reviews_unnested['sentiment_analysis'] = df_reviews_unnested['sentiment_polarity'].apply(lambda x: 0 if x < 0 else 1 if x == 0 else 2)

# Drop the intermediate 'cleaned_review' and 'sentiment_polarity' columns
df_reviews_unnested.drop(['cleaned_review', 'sentiment_polarity'], axis=1, inplace=True)

# Display the updated DataFrame
df_reviews_unnested.head()


,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review_text,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet great replayability In opinion zomb...,2
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,Its unique worth playthrough,2
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere The gunplay bit chunky times ...,2
3,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know think see title Barbie Dreamhouse Party...,2
4,js41637,http://steamcommunity.com/id/js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For simple actually simple truck driving Simul...,0


In [23]:
df_reviews_unnested['recommend'] = df_reviews_unnested.recommend.map({True : 1, False : 0})

In [24]:
# drop columns
df_reviews_unnested.drop(['user_url', 
                     'funny', 
                     'last_edited', 
                     'helpful', 
                     'review_text' ], axis=1, inplace=True)

In [27]:
# Drop rows with NaN values in the 'posted' column
df_reviews_unnested.dropna(subset=['posted'], inplace=True)

# Extract the year from the 'posted' column
df_reviews_unnested['posted_year'] = df_reviews_unnested['posted'].str.extract(r'(\d{4})')

#drop the posted column
df_reviews_unnested.drop(columns='posted', inplace=True)
# Display the updated DataFrame
df_reviews_unnested.head()


,user_id,item_id,recommend,sentiment_analysis,posted_year
0,76561197970982479,1250,1,2,2011
1,76561197970982479,22200,1,2,2011
2,76561197970982479,43110,1,2,2011
3,js41637,251610,1,2,2014
4,js41637,227300,1,0,2013


In [28]:
#save the dataset
df_reviews_unnested.to_parquet('../data/fixed/reviews.parquet')

# australian_user_items.json

In [29]:
# load the file
rows = []
with open('../data/raw/australian_users_items.json', encoding='MacRoman') as f:
    for line in f.readlines():
        rows.append(ast.literal_eval(line))
        
df_items = pd.DataFrame(rows)
df_items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [33]:
df_items.info()

<class 'pandas.core.frame.DataFrame'>
Index: 70912 entries, 0 to 88308
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      70912 non-null  object
 1   items_count  70912 non-null  int64 
 2   steam_id     70912 non-null  object
 3   user_url     70912 non-null  object
 4   items        70912 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.2+ MB


In [31]:
# drp duplicates and cast the user_id as string
df_items.drop_duplicates(subset="user_id", inplace=True)
df_items['user_id'] = df_items['user_id'].astype(str) 

In [32]:
# if the the list in items is empty then we eliminate them
df_items = df_items[df_items['items'].apply(lambda x: x != [])]

In [34]:
# unnest the json objects in the column items
items_des = []

for index, row in df_items.iterrows():
    user_id = row['user_id']
    items_count = row['items_count']
    steam_id = row['steam_id']
    user_url = row['user_url']
    items = row['items']
    
    for i in items:   
        new_row = {
        'user_id': user_id,
        'items_count': items_count,
        'steam_id' : steam_id,
        'user_url' : user_url,
        'item_id': i.get('item_id', ''),
        'item_name': i.get('item_name', ''),
        'playtime_forever': i.get('playtime_forever', ''),
        'playtime_2weeks': i.get('playtime_2weeks', '')
        }
        
        items_des.append(new_row)

df_items_unnested = pd.DataFrame(items_des)

In [35]:
# drop the columns that we are not going to use
df_items_unnested.drop(columns=['items_count',
                           'steam_id',
                           'user_url',
                           'item_name',
                           'playtime_2weeks'], inplace=True)

In [36]:
df_items_unnested =  df_items_unnested[df_items_unnested['playtime_forever'] != 0]

In [37]:
df_items_unnested.reset_index(drop=True, inplace=True)

In [38]:
# save the file
df_items_unnested.to_parquet('../data/fixed/items.parquet', engine='pyarrow')